In [6]:
!unzip /content/pytorch-diffusion-master.zip -d /content/


Archive:  /content/pytorch-diffusion-master.zip
bca8c5aaafd5a0e5ee90549c211b9d0119d50c3a
   creating: /content/pytorch-diffusion-master/
  inflating: /content/pytorch-diffusion-master/.gitignore  
  inflating: /content/pytorch-diffusion-master/LICENSE  
  inflating: /content/pytorch-diffusion-master/README.md  
  inflating: /content/pytorch-diffusion-master/data.py  
  inflating: /content/pytorch-diffusion-master/entry.ipynb  
   creating: /content/pytorch-diffusion-master/imgs/
  inflating: /content/pytorch-diffusion-master/imgs/cifar.gif  
  inflating: /content/pytorch-diffusion-master/imgs/fashion.gif  
  inflating: /content/pytorch-diffusion-master/imgs/mnist.gif  
  inflating: /content/pytorch-diffusion-master/model.py  
  inflating: /content/pytorch-diffusion-master/modules.py  


In [8]:
!pip install pytorch_lightning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 15.1 MB/s eta 0:00:00


In [12]:
%cd /content/pytorch-diffusion-master


/content/pytorch-diffusion-master


In [13]:
# Diffusion Model
import os,sys
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import numpy
from data import DiffSet
from model import DiffusionModel
from torch.utils.data import DataLoader

os.getcwd()

'/content/pytorch-diffusion-master'

In [14]:
# Forward Diffusion Process
diffusion_steps=100
dataset_choice='MNIST'
max_epoch=0
batch_size=1

load_model=False #tracks whether the model needs to be loaded from a checkpoint: set to False: instantiate the model for first time

load_version_num=1

In [15]:
#  checkpoint the code dring training process: training can resume the checkpoint
pass_version=None
last_checkpoint=None

if load_model:
  pass_version=load_version_num
  last_checkpoint=glob.glob(
      f"./lightning logs/{dataset_choice}/version_{load_version_num}/checkpoints/*.ckpt"
  )[-1]

In [16]:
# training dataset
train_dataset=DiffSet(True,dataset_choice)
val_dataset=DiffSet(False,dataset_choice)

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 21618728.20it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 42895075.72it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49151011.65it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 5095086.59it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [17]:
# instantiate our diffusion model- load model from a checkpoint if we have a previously trained model or we instantiate a brand new model model if no previous checkpoint is available
if load_model:
  model=DiffusionModel.load_from_checkpoint(last_checkpoint, in_size=train_dataset.size*train_dataset.size, t_range=diffusion_steps,img_depth=train_dataset.depth)

else:
  model=DiffusionModel(train_dataset.size*train_dataset.size, diffusion_steps, train_dataset.depth)



In [18]:
tb_logger=pl.loggers.TensorBoardLogger('lightning_logs/',name=dataset_choice,version=pass_version,)

trainer = pl.Trainer(max_epochs=max_epoch, log_every_n_steps=10, logger=tb_logger)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model,train_loader,val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name  | Type       | Params
--------------------------------------
0  | inc   | DoubleConv | 37.7 K
1  | down1 | Down       | 295 K 
2  | down2 | Down       | 1.2 M 
3  | down3 | Down       | 2.4 M 
4  | up1   | Up         | 6.2 M 
5  | up2   | Up         | 1.5 M 
6  | up3   | Up         | 406 K 
7  | outc  | OutConv    | 65    
8  | sa1   | SAWrapper  | 395 K 
9  | sa2   | SAWrapper  | 395 K 
10 | sa3   | SAWrapper  | 99.6 K
--------------------------------------
12.9 M    Trainable params
0         Non-trainable params
12.9 M    Total params
51.676    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [86]:
gen_samples = []
x = torch.arange(model.t_range, 0, -1).view(1, 1, -1, 1)
sample_steps = 100  # Adjust the value as needed

for t in range(sample_steps):
    t_tensor = torch.tensor(t, dtype=torch.float).view(1, 1)
    #x = model.denoise_sample(x, t_tensor)
    x = x.to(torch.long)  # Convert to Long data type
    if t % 50 == 0:
        gen_samples.append(x)


In [61]:
len(gen_samples)

2

In [77]:
gen_samples=torch.stack(gen_samples,dim=0).moveaxis(2,4).squeeze(-1)
gen_samples=(gen_samples.clamp(-1,1)+1)/2
gen_samples.shape

torch.Size([2, 1, 100, 1])

In [ ]:
gen_samples = torch.stack(tuple(gen_samples), dim=0).permute(0, 2, 3, 1)
gen_samples = (gen_samples.clamp(-1, 1) + 1) / 2

fig = plt.figure(figsize=(12, 6))
for i in range(gen_samples.shape[0]):
    plt.subplot(5, 6, i+1)
    plt.imshow((gen_samples[i, :, :, 0]*255).type(torch.uint8), cmap='gray')
    plt.axis('off')

plt.show()
